In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time
import itertools
import MetaTrader5 as mt5
import pytz
import matplotlib as py
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import arch
import math

## Connect to MetaTrader5

In [ ]:
initilized = mt5.initialize()

if initilized:
    print('Connected to MetaTrader5')
    print('Login:', mt5.account_info().login)
    print('Server:', mt5.account_info().server)

## Setting parameters and retrieving data

In [ ]:
sym = "XAUUSD"
timeframes = mt5.TIMEFRAME_M10
gmt_now = datetime.now(pytz.timezone('Etc/GMT'))
lot_size = 0.1

In [ ]:
#testing for 365 days which is 
raw = pd.DataFrame(mt5.copy_rates_from(sym, timeframes, gmt_now, 1000))
raw['time'] = pd.to_datetime(raw['time'], unit='s', errors='coerce')
#raw.drop(columns=['tick_volume', 'real_volume', 'high', 'low'], inplace=True)
#raw.set_index('time', inplace=True)

## Applying returns

In [ ]:
monte_ret = raw.copy()
monte_ret["pips"] = (monte_ret["close"] - monte_ret["close"].shift(1))
monte_ret = monte_ret.dropna()

# Percentage Returns
monte_ret["pct_returns"] = monte_ret["close"].pct_change().dropna()
monte_ret = monte_ret.dropna()

# Sort returns in ascending order
monte_ret_var = monte_ret.pct_returns

# Predicting volatility at time t+1 for monte carlo simulation
df = pd.DataFrame(monte_ret, columns=['pct_returns'])

# Rescale the returns data
df['scaled_returns'] = df['pct_returns'] * 10000


# GARCH(1,1) with rolling window = 2

rolling_window = 2
forecasts_variance = []  # Use forecasts_variance list to store the predicted variances
forecasts_mean = []  # Use forecasts_mean list to store the predicted means

for i in range(len(df['scaled_returns']) - rolling_window + 1):  # Note the +1 to ensure inclusive window

    # Slice the rescaled returns for the current rolling window
    windowed_returns = df['scaled_returns'].iloc[i : i + rolling_window]

    # Fit GARCH(1,1) model for the current window
    model = arch.arch_model(windowed_returns, vol='Garch', p=1, q=1)
    results = model.fit()

    # Forecast volatility and mean for the next time step
    forecast = results.forecast(horizon=1)
    forecast_variance = np.sqrt(forecast.variance.values[-1, -1]) / 10000  # Convert back to original scale
    forecast_mean = forecast.mean.values[-1, -1] / 10000  # Convert back to original scale

    forecasts_variance.append(forecast_variance)
    forecasts_mean.append(forecast_mean)

# Monte Carlo

In [ ]:
# Number of simulations
num_simulations = 200000

# Initialize arrays to store VaR results
var_95_mean = []
var_95_variance = []

# Perform Monte Carlo simulation and calculate 95% VaR for each forecasted mean and variance
for mean, variance in zip(monte_ret_var.shift(1), forecasts_variance):
    # Generate simulated returns for the current forecasted mean and variance
    simulated_returns = np.random.normal(mean, np.sqrt(variance), num_simulations)
    
    # Calculate the 95% VaR for the simulated return series
    var_95 = np.percentile(simulated_returns, 0.05)
    
    # Append VaR results to the arrays
    var_95_mean.append(var_95)

In [ ]:
monte_ret_2 = monte_ret.copy()
monte_ret_2.set_index("time")

checker = monte_ret_2.copy()  # Create a copy of monte_ret_2 DataFrame
checker = checker[1:]
# Assuming var_95_mean is a Series containing the VaR values

# Account for spreads
checker["low_spread"] = checker.low + checker.spread/100

checker['VaR_95'] = var_95_mean  # Add a new column 'VaR_95' to the checker DataFrame

# Determine price of VaR
checker['VaR_95_price'] = ((1-abs(checker.VaR_95)/200) / (1+abs(checker.VaR_95)/200)) * checker.low_spread

checker["pct_returns"] = checker["pct_returns"]

checker["HIT"] = (checker["low_spread"] < checker["VaR_95_price"].shift(1)).astype(int)

#pd.set_option('display.max_rows', 50)
print(pd.DataFrame(checker[["low_spread", "VaR_95_price", "HIT"]]))

In [ ]:
import mplfinance as mpf
import scipy.stats as stats
rate_parameter = checker['HIT'].mean()

# Define the number of occurrences for which you want to calculate the probability
k = 1  # Define the number of occurrences for which you want to calculate the probability

# Calculate the probability of k occurrences using the Poisson distribution
probability = stats.poisson.pmf(k, rate_parameter)
print("Probability of 1 occurrence:", probability)

In [ ]:
monte_ret_2.index = pd.to_datetime(monte_ret_2.time)
data = pd.DataFrame({
    'Open': monte_ret_2['open'],
    'High': monte_ret_2['high'],
    'Low': monte_ret_2['low'],
    'Close': monte_ret_2['close'],
}, index=monte_ret_2.index[1:])  # Exclude the first row to align with VaR and pct_returns

# Assuming 'pct_returns' is the column name containing the percentage returns data
pct_returns = checker['VaR_95_price']

# Plot the candlestick chart with the normalized VaR and percentage returns as additional plots
mpf.plot(data, type='candle', style='yahoo', addplot=[
    #mpf.make_addplot(var_95_mean, color='red', alpha=0.5),  # VaR as an additional plot
    mpf.make_addplot(pct_returns[:], color='green', alpha=0.5)  # Percentage returns as an additional plot
], title='Gold Chart with 95% VaR and % Returns',warn_too_much_data=10000)